# Метод параболы для уравнения $(x-1.1)^5=0$

In [ ]:
import numpy as np
import cmath
from math import comb

def make_coeffs(a: float, n: int) -> np.ndarray:
    coeffs = [comb(n, k) * (-a)**(n-k) for k in range(n+1)]
    coeffs = coeffs[::-1]
    return np.array(coeffs, dtype=np.complex128)

In [ ]:
def polyval(P: np.ndarray, x: complex) -> complex:
    y = 0+0j
    for a in P:
        y = y * x + a
    return y

def ras1(P: np.ndarray, x0: complex, x1: complex) -> complex:
    return (polyval(P, x1) - polyval(P, x0)) / (x1 - x0)

def ras2(P: np.ndarray, x0: complex, x1: complex, x2: complex) -> complex:
    return (ras1(P, x1, x2) - ras1(P, x0, x1)) / (x2 - x0)

def muller_step(P: np.ndarray, x0: complex, x1: complex, x2: complex) -> complex:
    f2 = polyval(P, x2)
    d1 = ras1(P, x1, x2)
    d2 = ras1(P, x0, x2)
    A  = ras2(P, x0, x1, x2)
    B  = d1 + d2 - ras1(P, x0, x1)
    D  = cmath.sqrt(B*B - 4*A*f2)
    denom = B + D if abs(B + D) > abs(B - D) else B - D
    return x2 - 2 * f2 / denom

def find_root_muller(P: np.ndarray, x_init: complex, eps: float, maxiter: int = 10000) -> complex:
    x0 = 0.85 * x_init 
    x1 = x_init
    x2 = 1.15 * x_init
    for _ in range(maxiter):
        x3 = muller_step(P, x0, x1, x2)
        if abs(polyval(P, x3)) < eps:
            return x3
        x0, x1, x2 = x1, x2, x3

def deflate(P: np.ndarray, root: complex) -> np.ndarray:
    n = P.size
    B = np.empty(n-1, dtype=np.complex128)
    B[0] = P[0]
    for i in range(1, n-1):
        B[i] = B[i-1] * root + P[i]
    return B


def mullers_all_roots(P: np.ndarray, eps: float = 1e-14) -> np.ndarray:
    roots = []
    while P.size > 1:
        r = find_root_muller(P, 1+0j, eps)
        roots.append(r)
        P = deflate(P, r)
    return np.array(roots, dtype=np.complex128)
    
P = make_coeffs(1.1, 5)
roots = mullers_all_roots(P)


In [61]:
print("Найденные корни:")
for r in roots:
    print(f" {r.real} + {r.imag}j")

Найденные корни:
 1.0986018710448326 + -0.0002006393671534958j
 1.0993771798790124 + 0.0012677792106174555j
 1.1012489071201006 + -0.0006595424020149749j
 1.1010132792358196 + 0.0009840111159257771j
 1.0997587627202339 + -0.001391608557374762j


In [56]:
print("Корень\t\t\tОстатки:")
for i in range(len(roots)):
    res = abs(polyval(P,roots[i]) - polyval(P, np.roots(P)[i]))
    print(f"{roots[i].real:+.10f}{roots[i].imag:+.10f}j\t{res:.2e}")

Корень			Остатки:
+1.0986018710-0.0002006394j	4.31e-15
+1.0993771799+0.0012677792j	4.31e-15
+1.1012489071-0.0006595424j	4.85e-15
+1.1010132792+0.0009840111j	4.74e-15
+1.0997587627-0.0013916086j	4.33e-15


In [5]:
x=np.roots(P)
for i in range(5):
    print(f"{roots[i].real:+.10f} {roots[i].imag:+.10f}j\t\t{x[i].real:+.10f} {x[i].imag:+.10f}j")

+1.0986018710 -0.0002006394j		+1.0996483094 +0.0012308353j
+1.0993771799 +0.0012677792j		+1.0987213454 +0.0000468806j
+1.1012489071 -0.0006595424j		+1.0995591947 -0.0012020374j
+1.1010132792 +0.0009840111j		+1.1010630103 +0.0007152808j
+1.0997587627 -0.0013916086j		+1.1010081402 -0.0007909594j


### Метод касательных

In [ ]:
import math
import numpy as np

NEWTON_TOL       = 1e-8
NEWTON_MAX_STEPS = 1000 
UNIQUE_TOL       = 1e-8


def polynomial_value(z: complex, coeffs: np.ndarray) -> complex:
    return np.polyval(coeffs, z)


def polynomial_derivative_value(z: complex, coeffs: np.ndarray) -> complex:
    return np.polyval(np.polyder(coeffs), z)

def deflate_polynomial(coeffs: np.ndarray, root: complex) -> np.ndarray:
    n = len(coeffs) - 1
    powers = root ** np.arange(n, dtype=np.complex128) 
    q = np.empty(n, dtype=np.complex128)
    for k in range(n):                                      
        q[k] = np.dot(coeffs[:k + 1], powers[k::-1])      
    return q

def find_roots(coeffs_real: list[float]) -> list[complex]:
    coeffs = np.asarray(coeffs_real, dtype=np.complex128)
    roots: list[complex] = []

    while len(coeffs) > 1:
        n = len(coeffs) - 1
        R = 1 + np.max(np.abs(coeffs[1:])) / abs(coeffs[0])
        if n == 1:
            r = 0.0
        else:
            r = 1 / (1 + np.max(np.abs(coeffs[1:-1])) / abs(coeffs[-1]))

        rings = 2 * n
        perim = 2 * n

        # сетка стартовых точек 
        for p in np.linspace(r, R, rings):
            for phi in np.linspace(0.0, 2.0 * math.pi, perim, endpoint=False):
                z_0 = p * (math.cos(phi) + 1j * math.sin(phi))
                z=z_0
                for _ in range(NEWTON_MAX_STEPS):
                    f  = polynomial_value(z, coeffs)
                    df = polynomial_derivative_value(z, coeffs)
                    if abs(df) < 1e-16:
                        break
                    z_next = z - f / df
                    if abs(z_next - z) < NEWTON_TOL:
                        z = z_next
                        break
                    z = z_next

                if not (math.isfinite(z.real) and math.isfinite(z.imag)):
                    continue
                if any(abs(z - r_0) < UNIQUE_TOL for r_0 in roots):
                    continue
                roots.append(z)
                coeffs = deflate_polynomial(coeffs, z)
                break
            else:
                continue
            break

    return roots





coeffs = [1.00000000000000, -808.710000000000, 246641.841200000, 
            -33898682.5068420, 1842011080.63283, -11822825758.0076, 34111432268.5231,
            -55344083765.8971, 54125863885.7217, -31826936635.5868, 10407310057.2455, -1459118284.25595]
roots = find_roots(coeffs)

print("Найдено корней:", len(roots))
for i, z in enumerate(roots, 1):
    print(f"root #{i}: {z}")

Найдено корней: 11
root #1: (0.9099999353886055+0j)
root #2: (0.9300003583438123+0j)
root #3: (0.9499994112987741+0j)
root #4: (1.0000019343534863+0j)
root #5: (1.0199956271341701+0j)
root #6: (1.030002851708563+0j)
root #7: (1.069999881772548+0j)
root #8: (200.00001236328276+0j)
root #9: (200.29995593822667+0j)
root #10: (200.5000369633223+0j)
root #11: (200.99999473516837+0j)


# Апроксимация числа Пи

In [ ]:
import mpmath as mp

def pi_chudnovsky(digits: int) -> mp.mpf:
    mp.mp.dps = digits + 10

    BASE         = mp.mpf(640320)
    BASE_CUBED   = BASE ** 3
    SQRT_BASE_C3 = mp.sqrt(BASE_CUBED)

    fact_6k = mp.mpf(1)
    fact_3k = mp.mpf(1)
    fact_k  = mp.mpf(1)                        

    series_sum = mp.mpf(0)
    max_k = digits // 14 + 2 

    for k in range(max_k + 1):
        linear_coeff = 13591409 + 545140134 * k
        numerator    = (-1)**k * fact_6k * linear_coeff
        denominator  = fact_3k * fact_k**3 * (SQRT_BASE_C3 * BASE_CUBED**k)
        series_sum  += numerator / denominator

        k_next = k + 1
        fact_6k *= mp.factorial(6 * k_next) / mp.factorial(6 * k)
        fact_3k *= (3 * k_next) * (3 * k_next - 1) * (3 * k_next - 2) / fact_3k
        fact_k  *= k_next

    inv_pi = 12 * series_sum
    return 1 / inv_pi                          # π

# ---------------- пример -----------------------------------------------------
d = 50
print(f"π (Чудновские)  {d} знаков:\n{pi_chudnovsky(d)}")


π (Чудновские)  50 знаков:
3.14159265358979323846264338174026386280491311773406633228319


In [ ]:
import time
import mpmath as mp

EXTRA = 40


def chudnovsky_iters_simple(digits: int, extra: int = EXTRA):
    old_dps = mp.mp.dps
    mp.mp.dps = digits + extra

    C = mp.mpf(640320)
    C3 = C ** 3
    sqrtC3 = mp.sqrt(C3)
    target = mp.mpf(10) ** (-digits)

    s = mp.mpf(0)
    k = 0
    start = time.perf_counter()

    while True:
        num = (-1) ** k * mp.factorial(6 * k) * (13591409 + 545140134 * k)
        den = mp.factorial(3 * k) * (mp.factorial(k) ** 3) * sqrtC3 * (C3 ** k)
        s += num / den

        pi_est = 1 / (12 * s)
        if abs(pi_est - mp.pi) < target:
            break
        k += 1

    elapsed = time.perf_counter() - start
    mp.mp.dps = old_dps 

    return k + 1, elapsed, pi_est


if __name__ == "__main__":
    print(f"{'цифры':>12} | {'итерации':>10} | {'среднее время'}")
    for d in (50, 100, 200, 500, 1000, 2000):
        it, t, _ = chudnovsky_iters_simple(d)
        print(f"{d:5d} знаков | {it:3d} итерац.| {t:.4f} сек")

       цифры |   итерации | среднее время
   50 знаков |   4 итерац.| 0.0002 сек
  100 знаков |   8 итерац.| 0.0005 сек
  200 знаков |  15 итерац.| 0.0005 сек
  500 знаков |  36 итерац.| 0.0019 сек
 1000 знаков |  71 итерац.| 0.0089 сек
 2000 знаков | 142 итерац.| 0.0398 сек


In [ ]:


import timeit
import mpmath as mp

EXTRA = 40
def machin_pi(digits: int) -> mp.mpf:
    old = mp.mp.dps
    mp.mp.dps = digits + EXTRA
    val = 4 * (4 * mp.atan(mp.mpf(1)/5) - mp.atan(mp.mpf(1)/239))
    mp.mp.dps = old
    return val             

digits_list = [50, 100, 200, 500, 1000]
repeats     = 10

print(f"{'digits':>7} | {'avg time, s':>12}")
print("-"*23)

for d in digits_list:
    times = timeit.repeat(
        stmt="machin_pi(d)",
        globals={"machin_pi": machin_pi, "d": d},
        number=1,
        repeat=repeats
    )
    avg_t = sum(times) / repeats
    print(f"{d:7d} | {avg_t:12.6f}")


 digits |  avg time, s
-----------------------
     50 |     0.000044
    100 |     0.000047
    200 |     0.000090
    500 |     0.000478
   1000 |     0.001954


In [ ]:
def machin_iters(digits: int) -> int:
    old = mp.mp.dps
    mp.mp.dps = digits + EXTRA

    x5, x239 = mp.mpf(1)/5, mp.mpf(1)/239
    term1, term2 = x5, x239
    sum1 = sum2 = mp.mpf(0)
    denom = 1
    target = mp.mpf(10) ** (-digits)

    iters = 0
    while True:
        sum1 +=  term1 / denom
        sum2 +=  term2 / denom
        pi_est = 4 * (4*sum1 - sum2)
        if abs(pi_est - mp.pi) < target:
            break
        denom += 2
        term1 *= -x5**2
        term2 *= -x239**2
        iters += 1

    mp.mp.dps = old
    return iters + 1         

digits_list = [50, 100, 200, 500, 1000, 2000]
repeats     = 10 

print(f"{'digits':>7} | {'avg time, s':>12} | {'iters':>6}")
print("-"*34)

for d in digits_list:
    times = timeit.repeat(
        stmt="machin_pi(d)",
        globals={"machin_pi": machin_pi, "d": d},
        number=1,
        repeat=repeats
    )
    avg_t = sum(times) / repeats

    k_iters = machin_iters(d)

    print(f"{d:7d} | {avg_t:12.6f} | {k_iters:6d}")

 digits |  avg time, s |  iters
----------------------------------
     50 |     0.000043 |     35
    100 |     0.000049 |     71
    200 |     0.000091 |    142
    500 |     0.000501 |    356
   1000 |     0.002020 |    714
   2000 |     0.006033 |   1429
